In [376]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import csv
import pandas as pd
import numpy as np
from openpyxl import Workbook
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException 
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.chrome.options import Options
import json
import re
import random
from datetime import date

In [377]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
#chrome_options.add_argument("headless")

In [378]:
##Dictionaries##
DbByPostcode = {
    'Energex':4215,
    'Ergon':4350,
    'Ausgrid':2000,
    'Endeavour':2115,
    'Essential Energy':2339,
    'Evoenergy':2600,
    'SA Power Networks': 5000,
    'TasNetworks':7000}

DbRetailers = {
   'Energex':'',
    'Ergon':'',
    'Ausgrid':'',
    'Endeavour':'',
    'Essential Energy':'',
    'Evoenergy':'',
    'SA Power Networks':'',
    'TasNetworks':''}

DbListOfPlans = {
    'Energex':[],
    'Ergon':[],
    'Ausgrid':[],
    'Endeavour':[],
    'Essential Energy':[],
    'Evoenergy':[],
    'SA Power Networks':[],
    'TasNetworks':[]}



In [384]:
def save_dict(dictionary,name):
    def set_default(obj):
        if isinstance(obj, set):
            return list(obj)
        raise TypeError
    x = date.today().strftime("%d-%m-%Y")
    with open("C:/Users/Blake/Desktop/Output/EnergyMadeEasyScrape/"+name+"_"+x+".json", 'w') as f: 
        json.dump(dictionary, default=set_default, sort_keys=True, fp = f )
    return('Successfully saved ' + name)

In [380]:
#Functions:
def GetRetailers(DbByPostcode, Type):
    """A function that scrapes start page of energy made easy and loops through input dic
    Args:
        DbByPostcode: A predefined dictionary of a single postcode in each distribution network
        Type: A string either resi (residential) or com (commercial)
        
    Returns:
        dictionary (DbRetailers): retailers in each distribution network
        """
    driver = webdriver.Chrome('C:/Users/Blake/Downloads/chromedriver.exe', options=chrome_options)
    for key,value in DbByPostcode.items():
        try:
               
            driver.get("https://www.energymadeeasy.gov.au/start")
            driver.find_element_by_name("electricity").click()
            driver.find_element_by_id("autocomplete-postcode").click()
            form = driver.find_element_by_id("autocomplete-postcode")
            form.send_keys(value)
            sleep(3)
            driver.find_element_by_class_name("autocomplete__results__item").click()
            if Type == "Resi":
                driver.find_element_by_name("1Person").click()
                driver.find_element_by_name("noUsage").click()
            if Type == "Com":
                driver.find_element_by_name("smallBusiness").click()
                driver.find_element_by_name("manual").click()


            sleep(3)
            driver.find_element_by_name("electricity-retailer").click()
            RawRetailers = driver.find_element_by_name("electricity-retailer").text
            RawRetailers = RawRetailers.split("\n")
            RawRetailers = RawRetailers[2:]
            dictrawretailers = dict.fromkeys(RawRetailers,)


            DbRetailers[key] = dictrawretailers
        except NoSuchElementException:
            print('NoSuchElementException')
    driver.close()
    return(DbRetailers)

In [390]:
def GetPlans(DbByPostcode,Type):
    """A Parent Function that Scrapes main results page on Energy Made Easy
    Args:
        DbByPostcode: A predefined dictionary of a single postcode in each distrubition network
        type: the type of premise to be scraped, either resi (Residential) or com (Commercial)

    Returns:
        dictionary (DbListOfPlans): links to each electricity plan by distributor
    """
    driver = webdriver.Chrome('C:/Users/Blake/Downloads/chromedriver.exe', options=chrome_options)
    for key,value in DbByPostcode.items():
        
        driver.get("https://www.energymadeeasy.gov.au/start")
        driver.find_element_by_name("electricity").click()
        driver.find_element_by_id("autocomplete-postcode").click()
        form = driver.find_element_by_id("autocomplete-postcode")
        form.send_keys(value)
        sleep(3)
        driver.find_element_by_class_name("autocomplete__results__item").click()
        
        ##Change of button options for Residential
        if Type == "Resi":
            #Completes Resi Form
            driver.find_element_by_name("1Person").click()
            driver.find_element_by_name("noUsage").click()
            sleep(0.5)
            driver.find_element_by_name('solarPanels-No').click()
            if len(driver.find_elements_by_name('pool-No')) > 0:
                driver.find_element_by_name('pool-No').click()
            sleep(0.5)
            if len(driver.find_elements_by_name('underfloorHeating-No')) > 0:
                driver.find_element_by_name('underfloorHeating-No').click()
            sleep(0.5)
            if len(driver.find_elements_by_name("gasMethod-Don't have gas")) > 0:
                driver.find_element_by_name("gasMethod-Don't have gas").click()

            driver.find_element_by_name('smartMeter-Not sure').click()
            driver.find_element_by_name('electricity-retailer').click()
            driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/div/div/div[3]/div[3]/div/section[2]/fieldset/div/select/option[2]").click()
            
        ##Change of button options for Residential
        if Type == "Com":
            #Completes Com Form
            driver.find_element_by_name("smallBusiness").click()
            driver.find_element_by_name("manual").click()
            sleep(0.5)
            driver.find_element_by_name('electricity-retailer').click()
            driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/div/div/div[3]/div[3]/div/section[1]/section/fieldset/div/select/option[2]").click()
            
            driver.find_element_by_name('solarPanels-No').click()
            if len(driver.find_elements_by_name('peakOffpeakRates-No')) > 0:
                driver.find_element_by_name('peakOffpeakRates-No').click()
            sleep(0.5)
            if len(driver.find_elements_by_name('controlledLoad-No')) > 0:
                driver.find_element_by_name('controlledLoad-No').click()
            sleep(0.5)
            driver.find_element_by_name('smartMeter-Not sure').click()
        
            driver.find_element_by_id('dateInputelectricity-bill-start-date').click()
            driver.find_element_by_xpath('/html/body/div/div[2]/div/main/div/div/div/div[3]/div[3]/div/section[3]/section/fieldset/div/div[1]/div/div[1]/div[1]/div/div[2]/div[2]/div[1]/div[2]/table/tbody/tr[1]/td[3]/button').click()
            driver.find_element_by_id('dateInputelectricity-bill-end-date').click()
            driver.find_element_by_xpath('/html/body/div/div[2]/div/main/div/div/div/div[3]/div[3]/div/section[3]/section/fieldset/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div[1]/div[2]/table/tbody/tr[1]/td[4]/button').click()
            InputUsage = driver.find_element_by_name('energyInputelectricityTotalUsage')
            InputUsage.click()
            InputUsage.send_keys(80)
            
        #Accept Terms and conditions:
        driver.find_element_by_class_name('image-checkbox__label').click()
        driver.find_element_by_class_name('btn.btn--green.btn--large.btn-compare-plans').click()
        
        #Waiting for url Change to start scrape:
        while driver.current_url == 'https://www.energymadeeasy.gov.au/start':
            if driver.current_url == 'https://www.energymadeeasy.gov.au/results':
                break
        sleep(1)
        driver.find_element_by_class_name('main-filters-button-desktop').click()

        driver.find_element_by_xpath("/html/body/div/div[2]/div/main/section/div[1]/div/div[1]/div[2]/div[2]/div[1]/div[6]/div[2]/div[1]/label").click()

        driver.find_element_by_xpath("/html/body/div/div[2]/div/main/section/div[1]/div/div[1]/div[2]/div[2]/div[1]/div[6]/div[2]/div[3]/label").click()
        driver.find_element_by_xpath("/html/body/div/div[2]/div/main/section/div[1]/div/div[1]/div[2]/div[2]/div[1]/div[6]/div[2]/div[3]/label").click()
        driver.find_element_by_id('applyFilters').click()
        driver.find_element_by_id('showAllPlans').click()
        while len(driver.find_elements_by_class_name("show-more-button.btn")) > 0:
            driver.find_element_by_class_name("show-more-button.btn").click()
            
        NumOfPlans = len(driver.find_elements_by_class_name("plan-results-tile"))+ 1
        Plans = []
        for x in range(1,NumOfPlans):
            if len(driver.find_elements_by_xpath("/html/body/div/div[2]/div/main/section/div[1]/div/div[3]/div[4]/div["+str(x)+"]/div/div[2]/div[1]/a")) > 0:
                Plans.append(driver.find_element_by_xpath("/html/body/div/div[2]/div/main/section/div[1]/div/div[3]/div[4]/div["+str(x)+"]/div/div[2]/div[1]/a").get_attribute("href"))

            if len(driver.find_elements_by_xpath("/html/body/div/div[2]/div/main/section/div[1]/div/div[3]/div[3]/div["+str(x)+"]/div/div[2]/div[1]/a")) > 0:
                Plans.append(driver.find_element_by_xpath("/html/body/div/div[2]/div/main/section/div[1]/div/div[3]/div[3]/div["+str(x)+"]/div/div[2]/div[1]/a").get_attribute("href"))

        RawRetailerLinks = Plans
        DbListOfPlans[key] = RawRetailerLinks 
    driver.close()

    return(DbListOfPlans)

In [394]:

def GetPlanDetails(DbListOfPlans, Type):
    """A Parent Function that Scrapes BIPD's (Basic Information Plan Documents) on the Energy Made Easy
    Args:
        DbListOfPlans: the return dictitonary of GetPlans function
        DbRetailers: the return dictitonary of GetRetailers function

    Returns:
        dictionary (EnergyMadeEasyPlans): BPID details by distrubtor and energy retailer
        """
    exists = "PlanDetails" in locals()
    if exists == True:
        del PlanDetails
    PlanDetails = {}
    
    for key,value in DbListOfPlans.items():
        driver = webdriver.Chrome('C:/Users/Blake/Downloads/chromedriver.exe', options=chrome_options)
        number = 0
        for url in value:
            sleep(random.randint(0, 3))
            driver.get(url)
            number = number + 1
            if number == 10:
                sleep(random.randint(0, 3))
                driver.get(url)
            sleep(2)
            try:
                Company_raw = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[1]/div/div[1]/h1").text
                sep = "-"
                Company = Company_raw.split(sep, 1)[0]
                Company = Company.strip()
            except NoSuchElementException:
                Company = False
                pass

            try:
                Logo = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[2]/div[1]/div[1]/img").get_attribute("src")
            except NoSuchElementException:
                Logo = False
                pass
            try:
                PlanName = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[2]/div[1]/div[2]/h2").text
            except NoSuchElementException:
                PlanName = False
                pass

            try:
                ID2 = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[2]/div[2]/div/div[1]/span/strong").text
                Plan_ID = ID2
            except NoSuchElementException:
                Plan_ID = False
            
            try:
                Website_Link = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[2]/div[2]/div/div[2]/a").get_attribute("href")
            
            except NoSuchElementException:
                Website_Link = False

            try:
                raw_phone = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[2]/div[2]/div/div[2]/div[1]").text
                if re.search('Call(.*)', raw_phone ) != None:
                    result = re.search('Call(.*)', raw_phone )
                    Company_Phone = result.group(1)
                    Company_Phone = Company_Phone.translate({ord(i):None for i in '$cents[]/'})
                    Company_Phone = Company_Phone.strip()
                
                raw_phone2 = driver.find_element_by_class_name("panel__contact").text
                if re.search('\nCall(.*)\n', raw_phone ) != None:
                    result = re.search('\nCall(.*)\n', raw_phone2 )
                    Company_Phone = result.group(1)
                    Company_Phone = Company_Phone.translate({ord(i):None for i in '$cents[]/'})
                    Company_Phone = Company_Phone.strip()
    
            except NoSuchElementException:
                Company_Phone = False

                #-----------------------Getting Electricity Plan Contract Details:------------------#
            try:
                if len(driver.find_elements_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[6]/div/div")) < 1:
                     driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[6]/div/button").click()
                if len(driver.find_elements_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[6]/div/div")) > 0:
                    #TODO
                    #contract_detailsraw = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[6]/div/div/div[1]/section").text
                    #sleep(0.5)
                    #if re.search('Contract length\n(.*)\n', contract_detailsraw) != None:
                    #    result = re.search('Contract length\n(.*)\n', contract_detailsraw)
                    #    Contract_Length = result.group(1)
                    #if re.search('Contract length(.*)\n', contract_detailsraw) != None:
                    #    result = re.search('Contract length(.*)\n', contract_detailsraw)
                    #    Contract_Length = result.group(1)
                    #sleep(0.5)
                    #if re.search('\nContract length(.*)\n', contract_detailsraw) != None:
                    #    result = re.search('\nContract length(.*)\n', contract_detailsraw)
                    #    Contract_Length = result.group(1)
                    #sleep(0.5)
                    #if len(Contract_Length)<3:
                    #    Contract_Length = "false"


                    contract_detailsraw = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[6]/div/div/div[1]/section").text
                    if re.search('Distributor\n(.*)\n', contract_detailsraw) != None:
                        Db = re.search('Distributor\n(.*)\n', contract_detailsraw)
                        Distributor = Db.group(1)
                        if len(Distributor)<3:
                            Distributor = False


            except NoSuchElementException:
                Distributor = False
                #Contract_Length = "False
                pass
            except ElementClickInterceptedException:
                pass

            try:
                Warnings = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[4]/div/div[1]/div[2]/section/section/ul").text
                Warnings = Warnings.split("\n")
            except NoSuchElementException:
                Warnings = False
                pass
            try:
                fees_raw = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[4]/div/div[1]/div/section").text
                if re.search('Membership fee per year\n(.*)\n', fees_raw) != None:
                    fee_result = re.search('Membership fee per year\n(.*)\n', fees_raw)
                    Annual_Fee = fee_result.group(1)
                    Annual_Fee = Annual_Fee.translate({ord(i):None for i in '$cents[]/'})
                    Annual_Fee = Annual_Fee.strip()
                else: Annual_Fee = "false"

            except NoSuchElementException:
                Annual_Fee= False
                pass
            try:
                fees_raw = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[4]/div/div[1]/div/section").text
                if re.search('Type 4 Meter Installation\n(.*)\n', fees_raw) != None:
                    fee_result = re.search('Type 4 Meter Installation\n(.*)\n', fees_raw)
                    One_Off_Fee = fee_result.group(1)
                    One_Off_Fee = One_Off_Fee.translate({ord(i):None for i in '$cents[]/'})
                    One_Off_Fee = One_Off_Fee.strip()

                else: One_Off_Fee = False

            except NoSuchElementException:
                One_Off_Fee = False
                pass
                #-----------------------Getting Electricity Plan Rates:------------------#

            try:
                pricing_raw = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[5]/div/div/div[2]/div[1]").text
                if re.search('\nGeneral usage rates(.*)cents/kWh\n', pricing_raw ) != None:
                    result = re.search('\nGeneral usage rates(.*)cents/kWh\n', pricing_raw )
                    Usage_Rate = result.group(1)
                    Usage_Rate = Usage_Rate.translate({ord(i):None for i in '$cents[]/'})
                    Usage_Rate = Usage_Rate.strip()
                    if len(str(Usage_Rate)) < 8:
                        if int(float(Usage_Rate)) > 0.001:
                            Usage_Rate = int(float(Usage_Rate)) / 100
                    else: Usage_Rate = False

                else: Usage_Rate = False

                if re.search('\nGeneral charges\nDaily supply charge(.*)cents/day', pricing_raw ) != None:
                    result = re.search('\nGeneral charges\nDaily supply charge(.*)cents/day', pricing_raw )
                    Supply_Charge = result.group(1)
                    Supply_Charge = Supply_Charge.translate({ord(i):None for i in '$cents[]/'})
                    Supply_Charge = Supply_Charge.strip()
                    if int(float(Supply_Charge)) > 0.001:
                        Supply_Charge = int(float(Supply_Charge)) / 100
            
                else: Supply_Charge = False


                if re.search('\nControlled load\n(.*)cents/day\n', pricing_raw ) != None:
                    result = re.search('\nControlled load\n(.*)cents/day\n', pricing_raw )
                    CL_Supply = result.group(1)
                    CL_Supply = CL_Supply.translate({ord(i):None for i in 'Daily supply charge$cents[]/'})
                    CL_Supply = CL_Supply.strip()
                    if int(float(CL_Supply)) > 0.001:
                        CL_Supply = int(float(CL_Supply)) / 100

                else: CL_Supply = False

                if re.search('Controlled load usage(.*)cents/kWh', pricing_raw ) != None:
                    result = re.search('Controlled load usage(.*)cents/kWh', pricing_raw )
                    CL1_Rate = result.group(1)
                    CL1_Rate = CL1_Rate.translate({ord(i):None for i in '$cents[]/'})
                    CL1_Rate = CL1_Rate.strip()
                    if len(str(CL1_Rate))<6:
                        if int(float(CL1_Rate)) > 0.001:
                            CL1_Rate = int(float(CL1_Rate)) / 100
                    
                    if len(str(CL1_Rate))>6:
                        CL1_Rate = CL1_Rate.strip()
                        rawcl = CL1_Rate.split("o")
                        CL1_Rate = rawcl[0].strip()
                        if int(float(CL1_Rate)) > 0.001:
                            CL1_Rate = int(float(CL1_Rate)) / 100
                        CL2_Rate = rawcl[1].strip()
                        if int(float(CL2_Rate)) > 0.001:
                            CL2_Rate = int(float(CL2_Rate)) / 100

                    else: CL2_Rate = False
                else: 
                    CL1_Rate = False
                    CL2_Rate = False

                if re.search('\nSolar feed-in\n(.*)cents/kWh exported', pricing_raw ) != None:
                    result = re.search('\nSolar feed-in\n(.*)cents/kWh exported', pricing_raw )
                    Solar_FiT = result.group(1)
                    Solar_FiT = Solar_FiT.translate({ord(i):None for i in '$cents[]/'})
                    Solar_FiT = Solar_FiT.strip()
                    if int(float(Solar_FiT)) > 0.001:
                        Solar_FiT = int(float(Solar_FiT)) / 100

                else: Solar_FiT = False

                if re.search('\nGreenPower\n(.*)', pricing_raw ) != None:
                    result = re.search('\nGreenPower\n(.*)', pricing_raw )
                    GreenPower = result.group(1)
                    GreenPower = GreenPower.strip()

                else: GreenPower = False     

            except NoSuchElementException:
                CL1_Rate= False
                CL_Supply = False
                CL2_Rate = False
                Supply_Charge = False
                Usage_Rate = False
                Solar_FiT = False
                GreenPower = False
                pass

            try:

                Discount = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[4]/div/div[1]/div[2]/section/ul/li/div/span[2]").text
                Discount = Discount.translate({ord(i):None for i in '%'})
                Discount = Discount.strip()
                if len(Discount) < 1:
                    Discount = False
            except NoSuchElementException:
                Discount = False

            
            if Supply_Charge != False:
                if len(str(Usage_Rate)) < 8:
                    DB = key

                    #-----------------------SAVING TO DICTIONARY------------------#
                    
                    
                    
                    if DB not in PlanDetails.keys():
                        PlanDetails[DB] = {}
                        
                    
                    if Company not in PlanDetails[DB].keys():
                        PlanDetails[DB][Company] = [{"PlanName":PlanName,
                                           "Plan_ID":Plan_ID,
                                           "Company_Phone":Company_Phone,
                                           "Website_Link":Website_Link, 
                                           "Logo":Logo,
                                           "Supply_Charge":Supply_Charge,
                                           "Usage_Rate": Usage_Rate,
                                           "Solar_FiT": Solar_FiT,
                                           "CL1_Rate": CL1_Rate,
                                           "CL2_Rate": CL2_Rate,
                                           "CL_Supply": CL_Supply,
                                           "Discount":Discount,
                                           "Annual_Fee":Annual_Fee,
                                           "One_Off_Fee":One_Off_Fee,
                                           "GreenPower": GreenPower,
                                           "Warnings": Warnings,
                                           "Inverter_Limit": False}]


                    if not any(k['Plan_ID'] == Plan_ID for k in PlanDetails[DB][Company]):
                        PlanDetails[DB][Company].append({"PlanName":PlanName,
                                           "Plan_ID":Plan_ID,
                                           "Company_Phone":Company_Phone,
                                           "Website_Link":Website_Link, 
                                           "Logo":Logo,
                                           "Supply_Charge":Supply_Charge,
                                           "Usage_Rate": Usage_Rate,
                                           "Solar_FiT": Solar_FiT,
                                           "CL1_Rate": CL1_Rate,
                                           "CL2_Rate": CL2_Rate,
                                           "CL_Supply": CL_Supply,
                                           "Discount":Discount,
                                           "Annual_Fee":Annual_Fee,
                                           "One_Off_Fee":One_Off_Fee,
                                           "GreenPower": GreenPower,
                                           "Warnings": Warnings,
                                           "Inverter_Limit": False})
                    print('Successfully Scraped, ' + Plan_ID + ' on the '+DB+' network')

    driver.close()
        
    return(PlanDetails)

In [385]:
save_dict(ResiDbRetailers,"ResiDbRetailers")

'Successfully saved ResiDbRetailers'

In [396]:

##Third Step: Run Functions

#Running GetRetailers Function
#ResiDbRetailers = GetRetailers(DbByPostcode,"Resi")
#ComDbRetailers = GetRetailers(DbByPostcode,"Com")
#Saving Dictionaries
#save_dict(ResiDbRetailers,"ResiDbRetailers")
#save_dict(ComDbRetailers,"ComDbRetailers")

#Running GetPlans Function
#ResiDbListOfPlans = GetPlans(DbByPostcode,"Resi")
#ComDbListOfPlans = GetPlans(DbByPostcode,"Com")
#Saving Dictionaries
#save_dict(ResiDbListOfPlans,"ResiDbListOfPlans")
save_dict(ComDbListOfPlans,"ComDbListOfPlans")

#Running GetPlansDetails Function
#ResiPlanDetails = GetPlanDetails(ResiDbListOfPlans, "Resi")
ComPlanDetails = GetPlanDetails(ComDbListOfPlans, "Com")
#Saving Dictionaries
#save_dict(ResiPlanDetails,"ResiPlanDetails")
#save_dict(ComPlanDetails,"ComPlanDetails")

Successfully Scraped, 1ST80756SBE1 on the Energex network
Successfully Scraped, 1ST80757SBE1 on the Energex network
Successfully Scraped, 1ST80758SBE1 on the Energex network
Successfully Scraped, 1ST82916MBE1 on the Energex network
Successfully Scraped, 1ST82917MBE1 on the Energex network
Successfully Scraped, 1ST82918MBE1 on the Energex network
Successfully Scraped, 1ST82919MBE1 on the Energex network
Successfully Scraped, AGL15126MBE3 on the Energex network
Successfully Scraped, AGL15147MBE3 on the Energex network
Successfully Scraped, AGL15148MBE3 on the Energex network
Successfully Scraped, AGL15173SBE3 on the Energex network
Successfully Scraped, AGL15175SBE3 on the Energex network
Successfully Scraped, ALI13042MBE3 on the Energex network
Successfully Scraped, ALI13058MBE3 on the Energex network
Successfully Scraped, ALI13076MBE3 on the Energex network
Successfully Scraped, ALI13087MBE3 on the Energex network
Successfully Scraped, ALI13098MBE3 on the Energex network
Successfully S

KeyboardInterrupt: 

In [393]:
ComDbListOfPlans

{'Energex': ['https://www.energymadeeasy.gov.au/plan?id=1ST80755SBE1&postcode=4215',
  'https://www.energymadeeasy.gov.au/plan?id=1ST80756SBE1&postcode=4215',
  'https://www.energymadeeasy.gov.au/plan?id=1ST80757SBE1&postcode=4215',
  'https://www.energymadeeasy.gov.au/plan?id=1ST80758SBE1&postcode=4215',
  'https://www.energymadeeasy.gov.au/plan?id=1ST82916MBE1&postcode=4215',
  'https://www.energymadeeasy.gov.au/plan?id=1ST82917MBE1&postcode=4215',
  'https://www.energymadeeasy.gov.au/plan?id=1ST82918MBE1&postcode=4215',
  'https://www.energymadeeasy.gov.au/plan?id=1ST82919MBE1&postcode=4215',
  'https://www.energymadeeasy.gov.au/plan?id=AGL15125MBE3&postcode=4215',
  'https://www.energymadeeasy.gov.au/plan?id=AGL15126MBE3&postcode=4215',
  'https://www.energymadeeasy.gov.au/plan?id=AGL15147MBE3&postcode=4215',
  'https://www.energymadeeasy.gov.au/plan?id=AGL15148MBE3&postcode=4215',
  'https://www.energymadeeasy.gov.au/plan?id=AGL15173SBE3&postcode=4215',
  'https://www.energymadeea

In [355]:
driver = webdriver.Chrome('C:/Users/Blake/Downloads/chromedriver.exe', options=chrome_options)

In [370]:
raw_phone = driver.find_element_by_class_name("panel__contact").text

if re.search('\nCall(.*)\n', raw_phone ) != None:
    result = re.search('\nCall(.*)\n', raw_phone )
    Phone = result.group(1)
    Phone = Phone.translate({ord(i):None for i in '$cents[]/'})
    Phone = Phone.strip()
Phone

'13 76 66'

In [364]:
try:
    Website_Link = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[2]/div[2]/div/div[2]/a").get_attribute("href")
except NoSuchElementException:
    Website_Link = False

In [365]:
Website_Link

'https://sonnen.com.au/energy'

In [366]:
raw_phone = driver.find_element_by_xpath("/html/body/div/div[2]/div/main/div/section/div/div[1]/div[2]/div[2]/div/div[2]/div[1]").text
if re.search('Call(.*)', raw_phone ) != None:
    result = re.search('Call(.*)', raw_phone )
    Phone = result.group(1)
    Phone = Phone.translate({ord(i):None for i in '$cents[]/'})
    Phone = Phone.strip()
Phone

'13 88 60'